In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import utils
import tqdm
from torch import nn
import pickle

torch.manual_seed(10)
device = 'cuda' if torch.cuda.is_available() else 'cpu'


metric = 'capex'
inputlength = 8

In [ ]:
def mm_norm(arr, normP):
    """
    Min max normalisation
    """
    arrmax = normP[0] 
    arrmin = normP[1]
    return (arr - arrmin)/(arrmax - arrmin)

def mm_rev(norm, normP):
    """
    Reverse min max normalisation
    """
    arrmax = normP[0] 
    arrmin = normP[1]
    return norm*(arrmax - arrmin) + arrmin

with open('device.txt', 'w') as f:
    f.write(device)
    f.write(f'\n{torch.cuda.is_available()}')


def save_pkl(data, save_name):
    """
    Saves .pkl file from of data in folder: tmp/
    """
    with open(save_name, 'wb') as handle:
        pickle.dump(data, handle, protocol = pickle.HIGHEST_PROTOCOL)
    #print(f'File saved at: {save_name}')
    return None

def load_pkl(file_name):
    """
    Loads .pkl file from path: file_name
    """
    with open(file_name, 'rb') as handle:
        data = pickle.load(handle)
    print(f'Data loaded: {file_name}')
    return data

In [ ]:

class ReLUNet(torch.nn.Module):
    """
    ReLU neural network structure.
    1 hidden layers, 1 output layer.
    size of input, output, and hidden layers are specified
    """
    def __init__(self, n_input, n_hidden, n_output, num_layers):
        super().__init__()

        self.hidden_layers = torch.nn.ModuleList()
        for i in range(num_layers):
            if i == 0:
                self.hidden_layers.append(torch.nn.Linear(n_input, n_hidden))
            else:
                self.hidden_layers.append(torch.nn.Linear(n_hidden, n_hidden))


        self.output = nn.Linear(n_hidden, n_output)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        for layer in self.hidden_layers:
            x = self.relu(layer(x))
        x = self.output(x)
        return x


In [ ]:
def load_model(fn, train_set):
    """
    Load ANN model for inference
    """

    P = load_pkl(fn)
    SD = P['state_dict']
    
    structure = P['structure']
    
    
    inputs = train_set[0].shape[1]
    outputs = train_set[1].shape[1]
    m = ReLUNet(inputs, structure[2], outputs, structure[3]).to(torch.float64)
    m.load_state_dict(SD)

    return m

In [ ]:
from botorch.utils.transforms import normalize, unnormalize

def normalize_x(params, space):
    bounds = torch.tensor([var['domain'] for var in space]).to(params).t()
    params = normalize(params, bounds)
    return params

def unnormalize_x(params, space):
    bounds = torch.tensor([var['domain'] for var in space]).to(params).t()
    params = unnormalize(params, bounds)
    return params

def wrap_X(X, space):

    def _wrap_row(row):
        wrapped_row = {}
        for i, x in enumerate(row):
            wrapped_row[space[i]['name']] = x.item()
        
            if space[i]['type'] == 'discrete':
                wrapped_row[space[i]['name']] = int(np.round(x.item()))
        return wrapped_row
    
    wrapped_X = []
    for i in range(X.shape[0]):
        wrapped_X.append(_wrap_row(X[i]))
        
    return wrapped_X


def unwrap_X(parameters, space):

    X = torch.zeros(len(parameters), len(space),
                    dtype=torch.float64)
    for i, p in enumerate(parameters):
        x = [p[var['name']] for var in space]
        X[i] = torch.tensor(x, dtype=torch.float64)
        
    return X

In [ ]:
# LOAD data
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

space = [
    #{'name': 'flask_time', 'type': 'continuous', 'domain': (21600, 108000)},
    #{'name': 'seed_time', 'type': 'continuous', 'domain': (43200, 108000)},
    #{'name': 'seed_fed_batch', 'type': 'continuous', 'domain': (0.001636, 0.004908)},
    {'name': 'seed_batch_med', 'type': 'continuous', 'domain': (0.010225325, 0.030675975)},
    #{'name': 'seed_conversion', 'type': 'continuous', 'domain': (0.9, 0.98)},
    #{'name': 'main_time', 'type': 'continuous', 'domain': (64800, 144000)},
    {'name': 'fed_batch', 'type': 'continuous', 'domain': (0.0190875, 0.0572625)},
    {'name': 'batch_med', 'type': 'continuous', 'domain': (0.102253255, 0.306759765)},
    {'name': 'conversion', 'type': 'continuous', 'domain': (0.9, 0.98)},
    #{'name': 'solid_conc', 'type': 'continuous', 'domain': (50, 300)},
    {'name': 'res_vol', 'type': 'continuous', 'domain': (0.28903585,  0.86710755)},
    #{'name': 'equi1', 'type': 'continuous', 'domain': (0.01, 0.05)},
    {'name': 'dia1', 'type': 'discrete', 'domain': (10, 50)},
    #{'name': 'flush1', 'type': 'continuous', 'domain': (0.0015,  0.0045)},
    #{'name': 'equi2', 'type': 'continuous', 'domain': (0.01, 0.05)},
    {'name': 'dia2', 'type': 'discrete', 'domain': (10, 50)},
    {'name': 'flush2', 'type': 'continuous', 'domain': (0.00067,  0.002)},
    #{'name': 'failure', 'type': 'continuous', 'domain': (0.0, 0.1)},
]

space2 = [
    #{'name': 'flask_time', 'type': 'continuous', 'domain': (21600*0.9, 108000*1.1)},
    #{'name': 'seed_time', 'type': 'continuous', 'domain': (43200*0.9, 108000*1.1)},
    #{'name': 'seed_fed_batch', 'type': 'continuous', 'domain': (0.001636*0.9, 0.004908*1.1)},
    {'name': 'seed_batch_med', 'type': 'continuous', 'domain': (0.010225325*0.9, 0.030675975*1.1)},
    #{'name': 'seed_conversion', 'type': 'continuous', 'domain': (0.9*0.9, 0.98*1.1)},
    #{'name': 'main_time', 'type': 'continuous', 'domain': (64800*0.9, 144000*1.1)},
    {'name': 'fed_batch', 'type': 'continuous', 'domain': (0.0190875*0.9, 0.0572625*1.1)},
    {'name': 'batch_med', 'type': 'continuous', 'domain': (0.102253255*0.9, 0.306759765*1.1)},
    {'name': 'conversion', 'type': 'continuous', 'domain': (0.9*0.9, 0.98*1.1)},
    #{'name': 'solid_conc', 'type': 'continuous', 'domain': (50*0.9, 300*1.1)},
    {'name': 'res_vol', 'type': 'continuous', 'domain': (0.28903585*0.9,  0.86710755*1.1)},
    #{'name': 'equi1', 'type': 'continuous', 'domain': (0.01*0.9, 0.05*1.1)},
    {'name': 'dia1', 'type': 'discrete', 'domain': (10*0.9, 50*1.1)},
    #{'name': 'flush1', 'type': 'continuous', 'domain': (0.0015*0.9,  0.0045*1.1)},
    #{'name': 'equi2', 'type': 'continuous', 'domain': (0.01*0.9, 0.05*1.1)},
    {'name': 'dia2', 'type': 'discrete', 'domain': (10*0.9, 50*1.1)},
    {'name': 'flush2', 'type': 'continuous', 'domain': (0.00067*0.9,  0.002*1.1)},
    #{'name': 'failure', 'type': 'continuous', 'domain': (0.0*0.9, 0.1*1.1)},
]

# Load inputs and outputs from CSV files
input_data = pd.read_csv('inputs.csv')
output_data = pd.read_csv('outputs.csv')

output_data = output_data[[metric]]


X_train, X_test, y_train, y_test = train_test_split(input_data, 
                                                    output_data, 
                                                    test_size=0.2, # 20% test, 80% train
                                                    random_state=42) # make the random split reproducible


# Split data into train and test sets
Xtrain = X_train.to_numpy(dtype = 'float64')
Ytrain = y_train.to_numpy(dtype = 'float64')
Xtest = X_test.to_numpy(dtype = 'float64')
Ytest = y_test.to_numpy(dtype = 'float64')

# Min max normalisation
Xmax = np.array([var['domain'][1] for var in space])
Xmin = np.array([var['domain'][0] for var in space])

XnormP = (Xmax, Xmin)

normP = XnormP

scaler = StandardScaler()
Y_train_standardized = scaler.fit_transform(Ytrain)
Y_test_standardized = scaler.transform(Ytest)


train_set = (mm_norm(Xtrain, normP),Y_train_standardized)
test_set = (mm_norm(Xtest, normP), Y_test_standardized )

In [ ]:
def SuperPro2(p, train_set, normP, space):

    ann_loaded = load_model('ann_ACC_0.19_24480_0.0000_437_1.pkl', train_set)

    with torch.no_grad():
        y_pred =ann_loaded(p)
        y_pred = scaler.inverse_transform(y_pred)
       
    return torch.tensor(y_pred)


def superpro(parameters, train_set, normP, space):

    score = SuperPro2(parameters, train_set, normP, space)


    return score

In [ ]:
from botorch.models import SingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.fit import fit_gpytorch_model
from gpytorch.kernels import MaternKernel, RBFKernel
from botorch.models.transforms import Standardize



def initialize_model(X, y, GP=None, state_dict=None, *GP_args, **GP_kwargs):


    covar_module = MaternKernel(nu=2.5)


    if GP is None:
        GP = SingleTaskGP
        
    model = GP(X, y,  outcome_transform=Standardize(1), covar_module = covar_module, *GP_args, **GP_kwargs).to(X)

    mll = ExactMarginalLogLikelihood(model.likelihood, model)

    # load state dict if it is passed
    if state_dict is not None:
        model.load_state_dict(state_dict)
    return mll, model

In [ ]:
#6. Repeat from step 1
from botorch.optim import optimize_acqf
def bo_step(X, y, objective, bounds, GP=None, acquisition=None, q=1, state_dict=None, plot=False):

    
    mll, gp = initialize_model(X, y, GP=GP, state_dict=state_dict)
    fit_gpytorch_model(mll)
    
    # Create acquisition function
    acquisition = acquisition(gp)

    # Optimize acquisition function if y is not None

    candidate = optimize_acqf(
        acquisition, bounds=bounds, q=q,  inequality_constraints=None, num_restarts=50, raw_samples=1024
    )
       
    X = torch.cat([X, candidate[0]])
    y = torch.cat([y, objective(candidate[0])])

    if plot:
        utils.plot_acquisition(acquisition, X, y, candidate)
        
    return X, y, gp

In [ ]:
bounds_01 = torch.zeros(2, len(space), dtype=torch.float64)
bounds_01[1] = 1

In [ ]:

def get_best_params(params, scores, space):
    bounds = torch.tensor([var['domain'] for var in space]).to(params).t()
    params = unnormalize(params, bounds)
    
    best_idx = np.argmin(scores.cpu().numpy())
    
    return wrap_X(params[[best_idx]], space)[0]

In [ ]:
from botorch.acquisition import (ExpectedImprovement, PosteriorMean,
                                 ProbabilityOfImprovement,
                                 UpperConfidenceBound, qKnowledgeGradient)
from botorch.acquisition.max_value_entropy_search import qMaxValueEntropy
from botorch.sampling import SobolQMCNormalSampler
from botorch.acquisition.max_value_entropy_search import qMaxValueEntropy


cum_best_df = pd.DataFrame()
best_params_df = pd.DataFrame()

execution_times_df = pd.DataFrame()

execution_times = []


for i in range(0, 100,10) :
    print('Seed:', i, 'out of 100')
    torch.manual_seed(i)
    start_time = time.time()
    params = torch.tensor(Xtrain[:5,:])

    
    scores = torch.tensor(Ytrain[:5])


    state_dict = None

    budget = 100

    objective = lambda x: superpro(x, train_set, normP, space)

    # Initialize the counter for consecutive iterations without improvement
    no_improvement_count = 0
    consecutive_iterations_threshold = 50  # Threshold for consecutive iterations without improvement

    best_score = float('+inf')  # Initialize the best score to a very low number

    with tqdm.tqdm(total=budget) as bar:
        while len(scores) < budget:
            n_samples = len(scores)
        
            # Assuming the rest of your optimization code goes here and updates `scores`
            GP = SingleTaskGP
        
            acquisition = lambda gp: UpperConfidenceBound(gp, beta=15, maximize=False)
        

            params, scores, gp = bo_step(params, scores, objective, bounds_01, GP=GP, acquisition=acquisition)
                                     
     
            current_best_score = scores.min().item()  # Get the current best score
        
            # Check if there is an improvement
            if current_best_score < best_score:
                best_score = current_best_score  # Update the best score
                no_improvement_count = 0  # Reset the counter since there was an improvement
            else:
                no_improvement_count += 1  # Increment the counter
        
            # Terminate if the number of consecutive iterations without improvement
            # reaches the threshold
            if no_improvement_count >= consecutive_iterations_threshold:
                print("Termination criterion met: No improvement for 20 consecutive iterations.")
                break  # Terminate the loop
        
            bar.update(len(scores) - n_samples)

        cum_best = np.minimum.accumulate(scores.cpu().numpy())


        # Create a temporary DataFrame from `cum_best` and reindex `cum_best_df` to ensure it's long enough
        temp_df = pd.DataFrame({f'{i}': cum_best.squeeze()})
        cum_best_df = cum_best_df.reindex(index=range(max(len(cum_best_df), len(temp_df)))).assign(**temp_df)
        best_param = get_best_params(params, scores, space)
        best_param_values = list(best_param.values())
        best_params_df[f'{i}'] = pd.Series(best_param_values)


In [ ]:
#=============================================================RESULTS==============================================================
for key, value in best_param.items():
        print(f'{key}: {value}\n')

print('Optimum', scores.min().item())

utils.plot_convergence(params, scores, maximize=False)

In [ ]:
cum_best = np.minimum.accumulate(scores.cpu().numpy())
